---
<center><u><h1>Slack Chatbots</h1></u></center>
---

---

# What is chatbot?

A chatbot is a service, powered by rules and sometimes artificial intelligence, that you interact with via a chat interface. The chatbots based on rules, tend to be limited in functionality, and are as smart as they are programmed to be. On the other end, chatbots that use artificial intelligence, understands language, not just commands, and continuously gets smarter as it learns from conversations it has with people. The service could be any number of things, ranging from functional to fun, and it could live in any major chat product (Slack, Facebook Messenger, Text Messages, etc.)

Here are some examples of chatbots

* [WeatherBot](https://www.messenger.com/login.php?next=https%3A%2F%2Fwww.messenger.com%2Ft%2Fhiponcho%2F") will get you the weather whenever you ask
* [NewsBot](https://www.messenger.com/login.php?next=https%3A%2F%2Fwww.messenger.com%2Ft%2FCNN%2F) will update you with news
* [Scheduling bot](https://x.ai/) is a personal assistant, which schedules meetings for you
* In China there is a bot called [Xiaoice](https://en.wikipedia.org/wiki/Xiaoice), build by Microsoft, over 20 million people are talking with it.

# Slack Chatbots

[Slack](https://slack.com) Bot users have many of the same qualities as their human counterparts: they have profile photos, names, and bios, they exist in the team directory, they can be direct messaged or mentioned, they can post messages and upload files, and they can be invited to and kicked out of channels and private groups.


The biggest difference between bot users and regular users in Slack is that instead of interacting with a team via one of Slack's mobile or desktop apps, bot users are controlled programmatically via a bot user token that accesses one or more of [Slack's APIs](https://api.slack.com).


# Instruction of creation of a chatbot in Slack

# <u>Part I. Create a team</u>

Here we will set our Slack team up.

* First go to [slack.com](https://slack.com/)
* Enter your e-mail and click **Crete a new team**
![alt text](images/1.jpg)
* Now you have to check your inbox for an e-mail with a confirmation code.
* Enter the code to the form, then you will be redirected to the next step.
![alt text](images/2.jpg)
* Now fill in your name, last name, and the username. Then click **Continue to Password**
![alt text](images/3.jpg)
* Make yourself a password, and click **Continue to Team Info**
![alt text](images/4.jpg)
* Now fill info about your team, click **Continue to Group Name**
![alt text](images/5.jpg)
* Come up with a name for your group, click **Continue to Team Domain**
![alt text](images/6.jpg)
* Here you can change the domain for your team, but it is filled automatically for you. Click **Create Team**
* You will be asked if you wish to send invitaions, click **Skip**

# <u>Part II. Create a chatbot</u>
Now, when you have created your team, you can start creating your bot.

* First click on drop-down in upper left corner.
![alt text](images/8.jpg)
* Now click on **Apps & Integration**
![alt text](images/9.jpg)
* This will open a new tab, there click on **Build**
![alt text](images/10.jpg)
* Now click on **Make a Custom Integration**
![alt text](images/11.jpg)
* Click on **bots**
![alt text](images/12.jpg)
* Make a bot name, and click **Add Bot Integration**
![alt text](images/13.jpg)
* Here you will find API Token for your bot
![alt text](images/14.jpg)
* Now scroll down the page and click **Save Integration**
![alt text](images/15.jpg)
* Please, be aware, that Slack Tokens **should** be kept safe!

# Interaction with Slack using Python

First we will take a look on how to send and recieve messages using [SlackClient](https://github.com/slackapi/python-slackclient) python package. 

You have to install it, do it with
```
pip install slackclient
```

Now let's import our dependencies and set `SlackClient` up.
The documentation for SlackClient can be found [here](http://slackapi.github.io/python-slackclient/).

In [ ]:
from slackclient import SlackClient
# Paste the token here
# Generally it is a bad practice, to hardcode it,
# But it is ok for demonstrational purposes
access_token = "PUT YOUR BOT'S TOKEN HERE"
slack_client = SlackClient(access_token)

To send a message, you have to use
`api_call` function specifying the API method, channel to which you would like to send the message and text of message you want to send. Additional info for SlackClient's functions could be found [here](http://slackapi.github.io/python-slackclient/basic_usage.html).

In [ ]:
slack_client.api_call(
    "chat.postMessage",
    channel="#general",
    text="Hello from Python!")

Here is an exapmle of how will it look in Slack.
<img src="images/16.jpg">

Recieving messages is a bit different though. It requires history to be fetched. Here we will recieve the last message.

In [ ]:
# First you have to get a list of channels,
channels = slack_client.api_call("channels.list",
                                 exclude_archived = 1)
# and find the needed channel
# for example "general", and take it's ID
for channel in channels.get('channels'):
    if channel['name'] == 'general':
        channel_id = channel['id']

# Fetch last one message from channel history
message = slack_client.api_call("channels.history",
                                channel = channel_id,
                                count = 1)

print(message.get('messages')[0]['text'])

There are much more methods available for API calls, you can find them [here](https://api.slack.com/methods).

# Birthday Bot

Now, let's build a new bot called Birthday bot. We want it to be able to fetch data about the users from simple [SQLite](https://en.wikipedia.org/wiki/SQLite) database, and if somebody has a birthday today, bot will reach with congratulations. Our bot will also be able to add users with their birthdays to database.

In [ ]:
# Paste name of your bot here
BOT_NAME = 'birthday-bot'

Here we'll fetch list of all users from our Slack by `api_call` function with `"users.list"` method. Then we'll find our bot's ID in the list.

In [ ]:
users = slack_client.api_call("users.list")
if users.get('ok'):
    # retrieve all users so we can find our bot's ID
    members = users.get('members')
    for member in members:
        # check that 'name' word is in member and the name is our bot's name
        if 'name' in member and member.get('name') == BOT_NAME:
            print("Bot ID for '" + member['name'] + "' is " + member.get('id'))
            BOT_ID = member.get('id')
else:
    print("could not find bot user with the name " + BOT_NAME)

In [ ]:
# Define string of form "<@U3XKPG90X>"
# to be able to cut it from text inputs
AT_BOT = "<@" + BOT_ID + ">"

Here we will define some basic functions for our bot. Functions to work with database: add somebody to database, and read data. Function to "say" something to a specific channel.

First, let's create a function to add a user to database. It will connect to `users.db` SQLite database using Python's [sqlite3](https://docs.python.org/3.5/library/sqlite3.html) library. And using SQL queries create a table in the base if it doesn't exist and add a user into this table.

SQL Queries are quite simple. They are just like English.

* The query to create table is:
```
CREATE TABLE table_name (column_name column_type, column2_name column2_type,...)
```

* The query to insert a row is:
```
INSERT INTO table_name VALUES (value1, value2,...)
```

* Query to select rows from table is:
```
SELECT column_name, column2_name,... FROM table_name
```

* To use SQL with `sqlite3` in Python you have to first create a connection to database.
```
conn = sqlite3.connect("database_file_name.db")
```

* Create a cursor.
```
c = conn.cursor()
```

* All SQL commands are executed by cursor's `execute` method.
```
c.execute(" some sql query ")
```
You may find very good reference for SQLite in python [here](http://pythoncentral.io/introduction-to-sqlite-in-python/).

In [ ]:
import sqlite3
def add_to_db(name, date):
    try:
        # This function writes name and date of birth entry to database
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute("CREATE TABLE IF NOT EXISTS users (name text, date text)")
        c.execute("INSERT INTO users VALUES (?,?)", [name, date])
        conn.commit()
        # After any action connection should be closed
        conn.close()
        return True
    except:
        return False

In [ ]:
from datetime import datetime, date

# Let's add a user to our database,
add_to_db('jason','1986-1-29')
# And another one with birthday today. You may change the year if any
# Retrieve today's date
today = date.today()
add_to_db('eric', '1990-{0:02d}-{1:02d}'.format(today.month, today.day))

Now let's create a function that will get all users from our database, check if anybody has a birthday today, based on this the function generates some output message, to congratulate users, or to state that there are no birthdays today.

In [ ]:
def congratulate():
    # This function allows our bot to take a look on users' database, and if any user has a birthday today,  
    # it will make our bot to 'say' congratulations for that user    
    
    # Read our data from sqlite database
    try:
        # Try-except wrapping is used for the case when the database is empty.
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        data = c.execute("SELECT * FROM USERS").fetchall()
        conn.close()
    except:
        data = []
        
    # Retrieve today's date
    today = date.today()    
    
    # Define simple indicator, whether there are any birthdays today.
    birthdays = False    
    messages = ""
    for user in data:
        # datetime.strptime can convert a string with specific formatting to date
        check = datetime.strptime(user[1],"%Y-%m-%d").date()
        if check.day == today.day and check.month == today.month:
            birthdays = True
            message = "Happy Birthday, @" + user[0] + "! :tada:"
            messages += message + "\n"         
    
    if birthdays == False:
        return "Sadly there are no Birthdays today!"
    else:
        return messages

And here's a function to let our bot `say` something. It will connect to Slack Realtime Messaging, and post a message to a specific channel from our bot.

In [ ]:
def say(message, channel):
    # This function allows our bot to 'say' something. Function posts a message into the channel by bot
    if slack_client.rtm_connect():
        slack_client.api_call("chat.postMessage",
                              channel = channel,
                              as_user = "true:",
                              text = message)

Now let's add a function to parse Slack output. It will cut the text, removing our bot's name from it, leaving only command we want to give him.

In [ ]:
def parse_slack_output(slack_rtm_output):
    # This function will take Slack's messages
    for output in slack_rtm_output:
        # test if there is 'text' word and name of bot in the message
        if output and 'text' in output and AT_BOT in output['text']:
            # and return only the part of 'text' after the name of bot
            return output['text'].split(AT_BOT)[1].strip(), output['channel']
    # or it will return nothing in other cases
    return None, None

Now we need to make a function to extract name and date of birth from a string. We'll implement it simply by splitting the string by words, and taking the word following the 'name' word as a name, and the word following the 'birthday' word as a date of birth.

For example, for the string `"add user to database name john birthday 1980-10-1"` the function will return ('john', '1980-10-1')

In [ ]:
# This function will extract name and date of birth from text
def retrieve_name_bday(text):    
    words = text.split() # Split text by spaces
    # This is quite straightforward, if there is a word 'name', then the next entry is the name itself, same for date of birth
    for i in range(0, len(words)):
        if words[i] == 'name':
            name = words[i+1]
        elif words[i] == 'birthday':
            bday = words[i+1]
    return name, bday

# For example
print(retrieve_name_bday('name john birthday 1980-10-1'))

Now let's make a function to let our bot to react to commands. Our bot is able to react to few expressions. Depending on the input strings, our bot will react differently.

* The input strings containing `'add'` and `'user'` substrings would lead to call of `retrieve_name_bday` and `add_to_db` functions, which will extract name and date of birth and paste this info to database. The bot will `say` informational message about success of this operation.

* The input strings containing `'bday'` or `'birth'` would lead to call of `congratulate` function, thus making our bot to congratulate any user, who has birthday today, if any.
* The input string with `'help'` word will make our bot to `say` out the special help message about it's usage.
* In any another case, our bot will not be able to process and understand our command, and will `say` `'I dont get it, sorry!'` error message.



In [ ]:
# Help message about our bot
help_message = """My commands are quite simple:
call me with *bday* or *birth*, to congratulate all people who have birthday today!
call me with *add user name NAME birthday YYYY-MM-DD* to add new user to database, please, pay attention to words order
call me with 'help' to see this fancy help message."""


def act(text, channel):
    # This function is responsible for bot's reactions to our input
    try:
        # Try-except is used for cases, when text is None
        # We have to convert text to lowercase to ensure that all commands are processed correctly
        text = text.lower()
        if 'add' in text and 'user' in text:
            name, bday = retrieve_name_bday(text)
            if add_to_db(name, bday):
                say('Added ' + name.title() +" who is born at " + bday + " to db", channel)
        elif 'bday' in text or 'birth' in text:
            say(congratulate(),channel)
        elif 'help' in text:
            say(help_message, channel)
        else:
            say('I dont get it, sorry!',channel)
    except:
        pass

And here is the code to make our bot 'live'. This code will take messages from slack and parse them. Then our bot will react to these messages.

This is an **infinite** loop, you may stop it with Interrupt Kernel button. ![alt text](images/17.jpg)

In [ ]:
import time
# This is the delay which will define how often our bot reads slack's output
READ_WEBSOCKET_DELAY = 0.5

if slack_client.rtm_connect():
    while True:
        text, channel = parse_slack_output(slack_client.rtm_read())
        act(text, channel)
        time.sleep(READ_WEBSOCKET_DELAY)

Here are examples of Bot's work.

* Bot's reaction for a message with 'help' word.
![alt text](images/18.jpg)
* Bot's reaction for a message with 'add' and 'user' words.
![alt text](images/19.jpg)
* Bot's reaction for a message with 'bday' or 'birth' words.
![alt text](images/20.jpg)
* Bot's reaction for a message with some unknown command.
![alt text](images/21.jpg)

> # Exercise 1

> Make a function that will calculate user's age based on username. The data should be taken from database.

In [ ]:
# type your code here

> # Exercise 2

> Add a new action to `act` function that will allow our bot to answer question about Age.

In [ ]:
# type your code here